In [1]:
using Pkg; Pkg.activate(".")
using DataFrames, DataFramesMeta, Statistics, Base.Iterators

# Pkg.status("DataFrames")
# Pkg.status("DataFramesMeta")

  Activating environment at `E:\PP\git\DataWrangler.jl\Project.toml`


## Most promising @mod   (maybe @mod_ with @mount df)

In [44]:
# This function is probably the most poorly written feature of the code so far.
function fillmissing(d1::DataFrame, d2::DataFrame)
    for v in names(d1)
        if (!hasproperty(d2, v) || (unique(typeof.(d2[:, v])) == [Missing]))
            if ((any(unique(typeof.(d1[!, v])) .<: Number)) & (any(unique(typeof.(d1[!, v])) .<: String)))
                d2[!, v] .= Any[] # This doesn't work 
                d2[!, v]  = allowmissing(d2[!, v])
                d2[:, v] .= missing
            elseif (any(unique(typeof.(d1[!, v])) .<: Number))
                d2[!, v] .= 0.0 # There must be a better way of doing this
                d2[!, v]  = allowmissing(d2[!, v])
                d2[:, v] .= missing
            elseif (any(unique(typeof.(d1[!, v])) .<: String))
                d2[!, v] .= "" # There must be a better way of doing this
                d2[!, v]  = allowmissing(d2[!, v])
                d2[:, v] .= missing
            else
                d2[!, v] .= missing
            end
        end
    end
    return d2
end

fillmissing (generic function with 1 method)

In [3]:
function pullmatches(args, regexlist)
    returnlist = fill(Expr[], length(regexlist) + 1)
    touched = fill(0, length(args))
    
    for (i, rx) in enumerate(regexlist)
        returnlist[i] = [v for v in args if occursin(rx, string(v)) ]
        touched += [occursin(rx, string(v)) for v in args]
    end
    
    returnlist[end] = [args[i] for i in 1:length(args) if touched[i]==0]
    
    return returnlist
    
end

pullmatches (generic function with 1 method)

In [4]:
macro mod(df, args...)
    
    iffs , bys , targets = pullmatches(args, [r"^(iff|when|\(:if\))\(", r"^(by|\(:by\))\("])
    
    (length(iffs) > 1) && 
        throw("Only one `iff()` allowed! Group multiple with `.&`.")
    (length(bys)  > 1) && 
        throw("Only one `by()`  allowed! Group multiple by with `,`.")
    
    (length(iffs) == 1) && (iffs[1].head = :(=); iffs[1].args[1] = :iff)
    
    groupbys = (length(bys) == 1) ? 
        (x -> (x isa QuoteNode ? eval(x) : x)).(bys[1].args[2:end]) : []

    d = copy(eval(df))

    (length(iffs) == 0) && (selector = fill(true, size(d, 1)))
    (length(iffs) == 1) && (selector = eval(:( (@transform $d $(iffs...)).iff) ))

    xout = d[selector, :]

    # If bys have been defined then group the dataframe by $bys
    (length(bys) == 1) && (xout = groupby(xout, [groupbys...]))

    xout = eval(:(  @transform $xout $(targets...) ))

    # If bys have been defined then ungroup the dataframe by $bys
    (length(bys) == 1) && (xout = DataFrame(xout))

    d = fillmissing(xout, d)

    d[selector, names(xout)] = xout

    d

end

@mod (macro with 1 method)

In [5]:
using DataFrames: _replace_columns!

macro mod!(x, args...)    
    quote
        y = (@mod $x $(args...))
        _replace_columns!($x, y)
    end
end

@mod! (macro with 1 method)

### Demo

In [6]:
n = 6
x = DataFrame(A=1:n, B=rand(1:2, n), C=rand(1:2, n))

,A,B,C
,Int64,Int64,Int64
1,1,1,2
2,2,1,2
3,3,2,1
4,4,1,2
5,5,2,1
6,6,2,2


In [7]:
@mod! x            D = rand(0:10, size(x,1))
@mod! x            D = (:C + :B) .* 10              (run = 1)  :if((2 .< :A) .& (:A .< 6))
@mod! x            E = round.(:A .^ .5, digits = 1) (run = 2)  :if(:B .< :A .- 1)
@mod! x :by(B, C)  H = round(mean(:A) , digits = 1) (run = 3)  :if(:A .> 3)

,A,B,C,D,run,E,H
,Int64,Int64,Int64,Float64?,Float64?,Float64?,Float64?
1,1,1,2,2.0,missing,missing,missing
2,2,1,2,8.0,missing,missing,missing
3,3,2,1,30.0,1.0,missing,missing
4,4,1,2,30.0,3.0,2.0,4.0
5,5,2,1,30.0,3.0,2.2,5.0
6,6,2,2,7.0,3.0,2.4,6.0


In [8]:
# Order doesn't matter
@mod! x  :if((2 .< :A) .& (:A .< 6))      (run=1)    (D = :C + :B)

,A,B,C,D,run,E,H
,Int64,Int64,Int64,Float64?,Float64?,Float64?,Float64?
1,1,1,2,2.0,missing,missing,missing
2,2,1,2,8.0,missing,missing,missing
3,3,2,1,3.0,1.0,missing,missing
4,4,1,2,3.0,1.0,2.0,4.0
5,5,2,1,3.0,1.0,2.2,5.0
6,6,2,2,7.0,3.0,2.4,6.0


In [9]:
# :by can take Symbols or string
@mod x   :if(:A .<= 3) :by(:B)    (H = -sum(:D))   (run = 5)  

,A,B,C,D,run,E,H
,Int64,Int64,Int64,Float64?,Float64?,Float64?,Float64?
1,1,1,2,2.0,5.0,missing,-10.0
2,2,1,2,8.0,5.0,missing,-10.0
3,3,2,1,3.0,5.0,missing,-3.0
4,4,1,2,3.0,1.0,2.0,4.0
5,5,2,1,3.0,1.0,2.2,5.0
6,6,2,2,7.0,3.0,2.4,6.0


### DocStrings

In [10]:
@doc raw"""
     pullmatches(args, regexlist)

args: A `Tuple{Expr, Expr...}` to be broken into different `Tuples` 
  if `string(expression)` contains the regex match.
regexlist: A list of regex syntax used used to select which expression belongs where.

returns a `List` of `Vector{Expr}`s equal in length to regexlist plus 1.

#### Example:
```
julia> macro demo(args...)
         regexlist = [r"^iff\(", r"^by\("]
         iffs , bys, other = pullmatches(args, regexlist) 
         @show iffs 
         @show bys
         @show other
       end

@demo (macro with 1 method)

julia> @demo iff(:P) iff(:T) by(:a + :b) (f = :a + :b) (g = :b + :b + b:b)

iffs = Expr[:(iff(:P)), :(iff(:T))]
bys = Expr[:(by(:a + :b))]
other = Expr[:(f = :a + :b), :(g = :b + :b + b:b)]

```
"""
pullmatches = pullmatches

pullmatches

In [11]:
@doc """
     fillmissing(dfrom::DataFrame, dto::DataFrame)

Take a `DataFrame` `dfrom` and use it as source to create columns in 
`dto` attempting to be aware (poorly) of type.

dfrom: DataFrame from
dto: DataFrame to



#### Example:
```
julia> dfrom = DataFrame(a = 1:3, b = "", c = [1, "", 1:1])
julia> dto = DataFrame(a = 1, d = "foo", e = 2, f= :A)
julia> fillmissing(dto, dfrom)

3×6 DataFrame
 Row │ a      b       c    d        e         f       
     │ Int64  String  Any  String?  Float64?  Missing 
─────┼────────────────────────────────────────────────
   1 │     1          1    missing  missing   missing 
   2 │     2               missing  missing   missing 
   3 │     3          1:1  missing  missing   missing 

# We can see from the example that fillmissing does not work with non-String/non-Number types.
```
"""
fillmissing = fillmissing

fillmissing

# Approaches to Data Cleaning and Management

#### DataWrangler.jl 

``` 
julia> observed = CSV.read("mydata.csv")

julia> first(observed, 5)
      5×4 DataFrame
       Row │ squarefoot  currency  year   cost    
           │ Int64       String    Int64  Float64 
      ─────┼──────────────────────────────────────
         1 │       1600  pesos      1965  16000.0
         2 │       1850  dollars    1938   1200.0
         3 │       2400  pesos      1904  19000.0
         4 │       1850  livre      1930  25500.0
         5 │       1000  dollars    1939    800.0

# Cost is based on currency conversions charts found on July x, xxxx
julia> @mod! observed  costUSD = :cost          :if( :currency .== "dollars" )
julia> @mod! observed  costUSD = :cost / 10     :if( :currency .== "pesos"   )
julia> @mod! observed  costUSD = :cost / 17     :if( :currency .== "livre"   )

julia> first(observed, 5)
     5×5 DataFrame
      Row │ squarefoot  currency  year   cost     costUSD  
          │ Int64       String    Int64  Float64  Float64? 
     ─────┼────────────────────────────────────────────────
        1 │       1600  pesos      1965  16000.0    1600.0
        2 │       1850  dollars    1938   1200.0    1200.0
        3 │       2400  pesos      1904  19000.0    1900.0
        4 │       1850  livre      1930  25500.0    1500.0
        5 │       1000  dollars    1939    800.0     800.0
```

#### DataWrangler.jl with @mod_ (not currently implemented)

``` 
julia> observed = CSV.read("mydata.csv")

julia> @mount observed
Subsequent @mod_ commands will now reference `observed`

# Cost is based on currency conversions charts found on July x, xxxx
julia> @mod_ :if( :currency .== "dollars")  costUSD = :cost          
julia> @mod_ :if( :currency .== "pesos"  )  costUSD = :cost / 10     
julia> @mod_ :if( :currency .== "livre"  )  costUSD = :cost / 17     

```

#### DataFrames.jl 

``` 
julia> observed = CSV.read("mydata.csv")

# Cost is based on currency conversions charts found on July x, xxxx
julia> observed.costUSD = ifelse.(observed.currency .== "dollars", observed.cost      , missing         )
julia> observed.costUSD = ifelse.(observed.currency .== "pesos"  , observed.cost ./ 10, observed.costUSD)
julia> observed.costUSD = ifelse.(observed.currency .== "livre"  , observed.cost ./ 17, observed.costUSD)
```

#### DataFramesMeta.jl 

``` 
julia> observed = CSV.read("mydata.csv")

# Cost is based on currency conversions charts found on July x, xxxx
julia> observed = @transform observed costUSD = ifelse.(:currency .== "dollars", :cost      ,  missing)
julia> observed = @transform observed costUSD = ifelse.(:currency .== "pesos"  , :cost ./ 10, :costUSD)
julia> observed = @transform observed costUSD = ifelse.(:currency .== "livre"  , :cost ./ 17, :costUSD)
```

#### DataFramesMeta.jl (with @transform! not yet in public version)

``` 
julia> observed = CSV.read("mydata.csv")

# Cost is based on currency conversions charts found on July x, xxxx
julia> @transform! observed costUSD = ifelse.(:currency .== "dollars", :cost      ,  missing)
julia> @transform! observed costUSD = ifelse.(:currency .== "pesos"  , :cost ./ 10, :costUSD)
julia> @transform! observed costUSD = ifelse.(:currency .== "livre"  , :cost ./ 17, :costUSD)
```

#### Stata 

Stata manages things from a single DataFrame at a time perspective. This creates a nice natural workflow when only one DataFrame is needed at a time.

``` 
$ load "mydata.dta"

* Cost is based on currency conversions charts found on July x, xxxx
$ gen       costUSD = cost          if currency == "dollars"
$ replace   costUSD = cost / 10     if currency == "pesos"
$ replace   costUSD = cost / 17     if currency == "livre"

```

### Generate Sample Data

In [12]:
using DataFrames, Random, DataFramesMeta
# using DataWrangler

In [36]:
# DataWrangler.jl
n = 10; Random.seed!(1234);
df = DataFrame(squarefoot    = rand(800:50:2500, n), 
               currency      = rand(["dollars", "pesos", "livre"], n),
               hiddenquality = .5 .+ rand(n),
               year          = rand(1900:2020, n))

# Create observed cost
@mod! df  yrfactor = (:year .- 1850) ./ 100
@mod! df  cost = 100  .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100)    :if(:currency .== "dollars")
@mod! df  cost = 1000 .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100)    :if(:currency .== "pesos")
@mod! df  cost = 1700 .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100)    :if(:currency .== "livre")
show(first(df, 5))

5×6 DataFrame
 Row │ squarefoot  currency  hiddenquality  year   yrfactor  cost     
     │ Int64       String    Float64        Int64  Float64?  Float64? 
─────┼────────────────────────────────────────────────────────────────
   1 │       1600  pesos          0.872846   1965      1.15   16000.0
   2 │       1850  dollars        0.763121   1938      0.88    1200.0
   3 │       2400  pesos          1.48869    1904      0.54   19000.0
   4 │       1850  livre          0.989858   1930      0.8    25500.0
   5 │       1000  dollars        0.925211   1939      0.89     800.0

In [36]:
# DataWrangler.jl (@mod_ not currently working)
n = 10; Random.seed!(1234);
df = DataFrame(squarefoot    = rand(800:50:2500, n), 
               currency      = rand(["dollars", "pesos", "livre"], n),
               hiddenquality = .5 .+ rand(n),
               year          = rand(1900:2020, n))

# Mount df for reference by objectless commands
@mount df

# Create observed cost
@mod_ yrfactor = (:year .- 1850) ./ 100
@mod_ cost = 100  .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100)    :if(:currency .== "dollars")
@mod_ cost = 1000 .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100)    :if(:currency .== "pesos")
@mod_ cost = 1700 .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100)    :if(:currency .== "livre")
show(first(df, 5))

5×6 DataFrame
 Row │ squarefoot  currency  hiddenquality  year   yrfactor  cost     
     │ Int64       String    Float64        Int64  Float64?  Float64? 
─────┼────────────────────────────────────────────────────────────────
   1 │       1600  pesos          0.872846   1965      1.15   16000.0
   2 │       1850  dollars        0.763121   1938      0.88    1200.0
   3 │       2400  pesos          1.48869    1904      0.54   19000.0
   4 │       1850  livre          0.989858   1930      0.8    25500.0
   5 │       1000  dollars        0.925211   1939      0.89     800.0

In [37]:
# With ifelse structure
n = 10; Random.seed!(1234);
df = DataFrame(squarefoot    = rand(800:50:2500, n), 
               currency      = rand(["dollars", "pesos", "livre"], n),
               hiddenquality = .5 .+ rand(n),
               year          = rand(1900:2020, n))

df.yrfactor = (df.year .- 1850) ./ 100

df.cost = ifelse.(df.currency .== "dollars", 100  .* round.(df.squarefoot .* df.yrfactor .*  df.hiddenquality ./ 100), missing)
df.cost = ifelse.(df.currency .== "pesos"  , 1000 .* round.(df.squarefoot .* df.yrfactor .*  df.hiddenquality ./ 100), df.cost)
df.cost = ifelse.(df.currency .== "livre"  , 1700 .* round.(df.squarefoot .* df.yrfactor .*  df.hiddenquality ./ 100), df.cost)
show(first(df, 5))

5×6 DataFrame
 Row │ squarefoot  currency  hiddenquality  year   yrfactor  cost    
     │ Int64       String    Float64        Int64  Float64   Float64 
─────┼───────────────────────────────────────────────────────────────
   1 │       1600  pesos          0.872846   1965      1.15  16000.0
   2 │       1850  dollars        0.763121   1938      0.88   1200.0
   3 │       2400  pesos          1.48869    1904      0.54  19000.0
   4 │       1850  livre          0.989858   1930      0.8   25500.0
   5 │       1000  dollars        0.925211   1939      0.89    800.0

In [41]:
# With DataFrames Meta @transform and ifelse structure
n = 10; Random.seed!(1234);
df = DataFrame(squarefoot    = rand(800:50:2500, n), 
               currency      = rand(["dollars", "pesos", "livre"], n),
               hiddenquality = .5 .+ rand(n),
               year          = rand(1900:2020, n))

df.yrfactor = (df.year .- 1850) ./ 100

df = @transform df cost = ifelse.(:currency .== "dollars", 100 .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100), missing)
df = @transform df cost = ifelse.(:currency .== "pesos",  1000 .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100), :cost)
df = @transform df cost = ifelse.(:currency .== "livre",  1700 .* round.(:squarefoot .* :yrfactor .*  :hiddenquality ./ 100), :cost)
show(first(df, 5))

5×6 DataFrame
 Row │ squarefoot  currency  hiddenquality  year   yrfactor  cost    
     │ Int64       String    Float64        Int64  Float64   Float64 
─────┼───────────────────────────────────────────────────────────────
   1 │       1600  pesos          0.872846   1965      1.15  16000.0
   2 │       1850  dollars        0.763121   1938      0.88   1200.0
   3 │       2400  pesos          1.48869    1904      0.54  19000.0
   4 │       1850  livre          0.989858   1930      0.8   25500.0
   5 │       1000  dollars        0.925211   1939      0.89    800.0

In [42]:
# Observed is a subset of 
observed = df[:, [:squarefoot, :currency,  :year, :cost]]
show(first(observed, 5))

5×4 DataFrame
 Row │ squarefoot  currency  year   cost    
     │ Int64       String    Int64  Float64 
─────┼──────────────────────────────────────
   1 │       1600  pesos      1965  16000.0
   2 │       1850  dollars    1938   1200.0
   3 │       2400  pesos      1904  19000.0
   4 │       1850  livre      1930  25500.0
   5 │       1000  dollars    1939    800.0

In [43]:
@mod! observed  costUSD = :cost          :if( :currency .== "dollars" )
@mod! observed  costUSD = :cost / 10     :if( :currency .== "pesos"   )
@mod! observed  costUSD = :cost / 17     :if( :currency .== "livre"   )
show(first(observed, 5))

5×5 DataFrame
 Row │ squarefoot  currency  year   cost     costUSD  
     │ Int64       String    Int64  Float64  Float64? 
─────┼────────────────────────────────────────────────
   1 │       1600  pesos      1965  16000.0    1600.0
   2 │       1850  dollars    1938   1200.0    1200.0
   3 │       2400  pesos      1904  19000.0    1900.0
   4 │       1850  livre      1930  25500.0    1500.0
   5 │       1000  dollars    1939    800.0     800.0

In [38]:
# Mount observed for modification by (@mod_ not currently working)
@mount observed
@mod_  costUSD = :cost          :if( :currency .== "dollars" )
@mod_  costUSD = :cost / 10     :if( :currency .== "pesos"   )
@mod_  costUSD = :cost / 17     :if( :currency .== "livre"   )
show(first(observed, 5))

5×5 DataFrame
 Row │ squarefoot  currency  year   cost     costUSD  
     │ Int64       String    Int64  Float64  Float64? 
─────┼────────────────────────────────────────────────
   1 │       1600  pesos      1965  16000.0    1600.0
   2 │       1850  dollars    1938   1200.0    1200.0
   3 │       2400  pesos      1904  19000.0    1900.0
   4 │       1850  livre      1930  25500.0    1500.0
   5 │       1000  dollars    1939    800.0     800.0